In [22]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.WARN)

In [23]:
import os
from datetime import datetime
import ray
from ray import tune
from ray.rllib.agents.ppo import PPOTrainer
from ray.tune.registry import register_env
from ray.tune.logger import CSVLogger, TBXLogger
from rllib_utils.env import launch_and_wrap_env
from rllib_utils.utils import seed
from rllib_utils.rllib_callbacks import on_episode_start, on_episode_step, on_episode_end, on_train_result
from rllib_utils.rllib_loggers import TensorboardImageLogger


# 1. Setting up Configs

In [24]:
seed = 42

## 1.1 Envirnoment config


In [25]:
environment_config = {
    # Run mode allows to load different settings in different run modes. Available options: 'train', 'inference', 'debug'
    "mode": 'debug',
    # Length of an episode (if not terminated due to a failure)
    "episode_max_steps": 500,
    # The input image will be scaled to (height, width)
    "resized_input_shape" : '(84, 84)',
    # Crop the top part of the image
    "crop_image_top": 'true',  # The top 1/top_crop_divider part of the image will be cropped off. (E.g. 3 crops the top third of the image)
    "top_crop_divider": 3,
    # Convert the image to grayscale
    "grayscale_image": 'false',
    # Stack multiple frames as input
    "frame_stacking": 'true',
    # Number of frames to stack if frame stacking is enabled
    "frame_stacking_depth": 3,
    # Apply motion blur to the images during training
    "motion_blur": 'false',
    # Map the action space to a certain type. Available options are:
    # 'leftright', leftright_clipped, 'leftright_braking', steering_braking, 'discrete'
    # 'heading', 'heading_smooth', 'heading_trapz', 'heading_sine', 'heading_limited',
    "action_type": 'heading',
    # Overwrite the default reward function of Gym Duckietown ('default' leaves the default reward of the gym)
    # Available options: 'default', 'default_clipped', 'posangle', 'lane_distance'
    "reward_function": 'posangle',
    # Set Gym Duckietown's distortion parameter to generate fisheye distorted images
    "distortion": 'true',
    # How large ange deviation should be accepted when the robot is placed into the simulator
    "accepted_start_angle_deg": 4,
    "simulation_framerate": 20,
    # Skip frames in the agent-environment loop and only step the environment using the last action
    "frame_skip": 1,
    # Computed actions come into effect this much later in the time period of a step.
    # Allowed values: floats in the (0., 1.) interval or 'random' to get random values in each instance of the env
    "action_delay_ratio": 0.0,
    # Map(s) used during training. Individual map names could be specified or 'multimap1' to train on a custom map set
    "training_map": 'udem1',
    # Use Gym Duckietown's domain randomization
    "domain_rand": 'false',
    "dynamics_rand": 'false',
    "camera_rand": 'false',
    # If >0.0 a new observation/frame will be the same as the previous one, with a probability of frame_repeating
    "frame_repeating" : 0.0,
    # Spawn obstacles (duckies, duckiebots, etc.) to random drivable positions on a map (with or without fixed obstacles)
    # To spawn other duckiebots this option is not recommended, use spawn_forward_obstacle instead
    # Type and amount of spawned obstacles is determined by the dictionary under the 'obstacles' key
    "spawn_obstacles": 'false',
    "obstacles" :{
      # Keys at this level specify the type of object to be spawned.
      # Supported options: 'duckie', 'duckiebot', 'cone', 'barrier'
      "duckie" :{
          # Density: Amount of ducks per drivable tile. Can't be larger than one currently
          "density": 0.5,
          # Non-static objects move according to their default behaviour implemented in the gym Duckietown
          # Duckies "walk" back and forth (like crossing the road), duckiebots perform lane following.
          "static": 'true'},
      "duckiebot" :{
          "density": 0,
          "static": 'false'}},
    # Spawn a duckiebot in front of the controlled robot in every episode
    # Parameters of the robot are randomised, but settings are hardcoded in ForwardObstacleSpawnnigWrapper)
    "spawn_forward_obstacle": 'false',
    # Evaluators of the AI driving olympics use small steps to generate many frames which are overlayed to get blured images
    # The aido wrapper implements this blur, and also implements the same dynamics simulation
    #Warning: Using AIDOWrapper slows down the environment simulation (and therefore the training)!
    # Computing an observation can take 10-20 times longer as without it!
    "aido_wrapper": 'false',
    # RLlib only allows unknown keys in the env config -> important "global" keys are kept/copied here
    "wandb":{
      "project": 'duckietown-rllib'},
    "experiment_name": 'experiment',
    "seed": "0000",}

## 1.2. RLlib config

In [26]:
ray_config = {
    "timesteps_total": 1.e+4,
    
    "ray_init_config" :{
#         "address": 127.0.0.1,
        "num_cpus": 17,
#         "webui_host": 127.0.0.1
    }
,
    # To load a trained model to continue training DO NOT USE THIS OPTION FOR PRETRAINING
    # -1 means no trained models are restored, training starts from random weights
    "restore_seed": -1,
    # If multiple trainings/experiments ran with the same seed
    "restore_experiment_idx": 0,
    # Every training saves two checkpoints the best (reward) and the final
    # Pay attention to trials with only one checkpoint (last = best)
    "restore_checkpoint_idx": 0,
    # For debugging on systems with less ram and vram
    "debug_hparams":{
      "rllib_config":{
        "num_workers": 1,
        "num_gpus": 0},
    #    train_batch_size: 64
    #    sgd_minibatch_size: 32
    #    eager: true
    #    log_level: 'DEBUG'
    #    num_sgd_iter: 2
      "ray_init_config":{
        "num_cpus": 1,
        "memory": 2097152000, # 2000 * 1024 * 1024
        "object_store_memory": 209715200, # 200 * 1024 * 1024
        "redis_max_memory": 209715200, # 200 * 1024 * 1024
        "local_mode": True}},


    "inference_hparams": {
      "rllib_config": {
        "explore": 'false',
        "num_workers": 0,
        "num_gpus": 0,
        "callbacks": {}},
      "ray_init_config":{
        "num_cpus": 1,
        "memory": 2097152000, # 2000 * 1024 * 1024
        "object_store_memory": 209715200, # 200 * 1024 * 1024
        "redis_max_memory": 209715200, # 200 * 1024 * 1024
        "local_mode": True}}

}

## 1.3 PPO config

In [27]:

ppo_config = {
    
  'env': 'Duckietown',
  'callbacks': {'on_episode_start': on_episode_start,
                'on_episode_step': on_episode_step,
                'on_episode_end': on_episode_end,
                'on_train_result': on_train_result},
  "env_config": environment_config,  
  # === RLlib common congfig ================================================
  # https://ray.readthedocs.io/en/latest/rllib-training.html#common-parameters
  # Number of rollout worker actors to create for parallel sampling. Setting
  # this to 0 will force rollouts to be done in the trainer actor.
  "num_workers": 8,
  # Default sample batch size (unroll length). Batches of this size are
  # collected from rollout workers until train_batch_size is met.
  "sample_batch_size": 265,
  # Number of GPUs to allocate to the trainer process. This can be fractional
  # (e.g., 0.3 GPUs).
  "num_gpus": 1,
  # Training batch size, if applicable. Should be >= sample_batch_size.
  # Samples batches will be concatenated together to a batch of this size,
  # which is then passed to SGD.
  "train_batch_size": 4096,
  # Discount factor of the MDP.
  "gamma": 0.99,
  # The default learning rate.
  # Note, that scientific notation is only interpreted as a number if the . and the sign are included!
  "lr": 5.e-5,
  # Whether to write episode stats and videos to the agent log dir. This is
  # typically located in ~/ray_results.
  "monitor": 'false',
  # Evaluate with every `evaluation_interval` training iterations.
  # The evaluation stats will be reported under the "evaluation" metric key.
  # Note that evaluation is currently not parallelized, and that for Ape-X
  # metrics are already only reported for the lowest epsilon workers.
  "evaluation_interval": None, # 25
  # Number of episodes to run per evaluation period. If using multiple
  # evaluation workers, we will run at least this many episodes total.
  "evaluation_num_episodes": 2,
  # Typical usage is to pass extra args to evaluation env creator
  # and to disable exploration by computing deterministic actions.
  # IMPORTANT NOTE: Policy gradient algorithms are able to find the optimal
  # policy, even if this is a stochastic one. Setting "explore=False" here
  # will result in the evaluation workers not using this optimal policy!
  "evaluation_config": {"monitor": 'true'},
  # This argument, in conjunction with worker_index, sets the random seed of
  # each worker, so that identically configured trials will have identical
  # results. This makes experiments reproducible.
  "seed": 1234,
  # === PPO-specific config =================================================
  # https://ray.readthedocs.io/en/latest/rllib-algorithms.html#proximal-policy-optimization-ppo
  # The GAE(lambda) parameter.
  "lambda": 0.95,
  # Total SGD batch size across all devices for SGD. This defines the
  # minibatch size within each epoch.
  "sgd_minibatch_size": 128,
  # Coefficient of the value function loss. IMPORTANT: you must tune this if
  # you set vf_share_layers: True. (it's False by default).
  "vf_loss_coeff": 0.5,
   # Coefficient of the entropy regularizer.
  "entropy_coeff": 0.0,
  # PPO clip parameter.
  "clip_param": 0.2,
  # Clip param for the value function. Note that this is sensitive to the
  # scale of the rewards. If your expected V is large, increase this.
  "vf_clip_param": 0.2,
  # If specified, clip the global norm of gradients by this amount.
  "grad_clip": 0.5
}

# 2. Training setup

- Set logger

- Initialize Ray for training

In [28]:
ray.init(num_cpus = ray_config["ray_init_config"]["num_cpus"])

Exception: Perhaps you called ray.init twice by accident? This error can be suppressed by passing in 'ignore_reinit_error=True' or by calling 'ray.shutdown()' prior to 'ray.init()'.

In [29]:
register_env('Duckietown', launch_and_wrap_env)

In [30]:
tune.run(PPOTrainer,
         stop={'timesteps_total': ray_config["timesteps_total"]},
         config=ppo_config,
         local_dir="./artifacts",
         checkpoint_at_end=True,
         trial_name_creator=lambda trial: trial.trainable_name,  # for PPO this will make experiment dirs start with PPO_
         name="ppo_DA",
         keep_checkpoints_num=1,
         checkpoint_score_attr="episode_reward_mean",
         checkpoint_freq=1,
         loggers=[CSVLogger, TBXLogger, TensorboardImageLogger],
#          verbose = 1,
         )

Trial name,status,loc
PPO,RUNNING,


(pid=3403) INFO:typing:version: 6.1.8
(pid=3403) DEBUG:duckietown_world:duckietown-world version 6.2.24 path /home/robertmoni/anaconda3/envs/sim2real_ad/lib/python3.6/site-packages
(pid=3403) INFO:geometry:version: 2.0.5
(pid=3403) DEBUG:aido_schemas:aido-protocols version 6.0.41 path /home/robertmoni/anaconda3/envs/sim2real_ad/lib/python3.6/site-packages
(pid=3403) INFO:nodes:version 6.2.4 path /home/robertmoni/anaconda3/envs/sim2real_ad/lib/python3.6/site-packages pyparsing 2.4.7
(pid=3403) 2021-07-13 08:11:48,365	INFO trainer.py:420 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
(pid=3403) 2021-07-13 08:11:48,366	INFO trainer.py:580 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=3403) INFO:duckietown_world: data: /home/robertmoni/anaconda3/envs/sim2real_ad/lib/python3.6/site-packages/duckietown_world/data
(pid=3403) /home/robertmoni/anaconda3/envs/sim2real_ad/lib/python3.6/si

2021-07-13 08:14:35,465	WARNING logger.py:221 -- You are trying to log an invalid value (ray/tune/hist_stats/_robot_coordinates=[[array([0.99494139, 0.72341046]), array([0.99494139, 0.72341046]), array([0.99494139, 0.72341046]), array([0.99682511, 0.72306437]), array([1.00308577, 0.72174442]), array([1.01243965, 0.71963559]), array([1.02312048, 0.7174147 ]), array([1.03479911, 0.71538734]), array([1.04717539, 0.71326371]), array([1.06004679, 0.71080776]), array([1.07404866, 0.70824525]), array([1.08878769, 0.70582038]), array([1.10465824, 0.70308118]), array([1.12128765, 0.69976807]), array([1.13907461, 0.69539385]), array([1.15715571, 0.69025357]), array([1.17455472, 0.68504594]), array([1.19124955, 0.67943199]), array([1.2072397 , 0.67318608]), array([1.2240092 , 0.66623184]), array([1.24146137, 0.65898554]), array([1.25897521, 0.65199369]), array([1.27598966, 0.64503241]), array([1.29215742, 0.63791426])], [array([0.94225525, 1.64985995]), array([0.94225525, 1.64985995]), array([0.9

Trial name,status,loc,reward,total time (s),ts,iter
PPO,RUNNING,152.66.246.136:3403,9.48305,147.867,4240,1


(pid=3563) WARNING:gym-duckietown:Could not find a valid starting pose after 5000 attempts
Result for PPO:
  episode_reward_max: 42.07679508058312
  episode_reward_min: -0.1878435093815226
  episode_reward_mean: 9.452859146791821
  episode_len_mean: 34.421487603305785
  episodes_this_iter: 121
  policy_reward_min: {}
  policy_reward_max: {}
  policy_reward_mean: {}
  custom_metrics:
    mean_robot_speed_mean: 0.2850800199664233
    mean_robot_speed_min: 0.0
    mean_robot_speed_max: 0.39591544036671145
    deviation_centerline_mean: 0.14777069040541235
    deviation_centerline_min: 0.017642635890195265
    deviation_centerline_max: 0.40962938889322686
    deviation_heading_mean: 19.867776199751734
    deviation_heading_min: 0.9129021046492997
    deviation_heading_max: 54.733138293859305
    distance_travelled_mean: 0.3230824564023807
    distance_travelled_min: 0.0
    distance_travelled_max: 2.1334741456505735
    distance_travelled_any_mean: 0.5723643022549488
    distance_travelled

...

Result for PPO:
  episode_reward_max: 60.62428815749328
  episode_reward_min: -2.0304892551444436
  episode_reward_mean: 9.883532770018313
  episode_len_mean: 34.92622950819672
  episodes_this_iter: 122
  policy_reward_min: {}
  policy_reward_max: {}
  policy_reward_mean: {}
  custom_metrics:
    mean_robot_speed_mean: 0.285871824847824
    mean_robot_speed_min: 0.0
    mean_robot_speed_max: 0.39804164361406763
    deviation_centerline_mean: 0.15431031983998822
    deviation_centerline_min: 0.005958838505413899
    deviation_centerline_max: 0.4830523125440354
    deviation_heading_mean: 17.366716634485634
    deviation_heading_min: 0.16172286452502785
    deviation_heading_max: 52.45996493420819
    distance_travelled_mean: 0.3210178153783877
    distance_travelled_min: 0.0
    distance_travelled_max: 1.520121090750308
    distance_travelled_any_mean: 0.5743611006856602
    distance_travelled_any_min: 0.0
    distance_travelled_any_max: 2.874516587088631
    proximity_penalty_mean: -0.

2021-07-13 08:19:07,014	WARNING logger.py:221 -- You are trying to log an invalid value (ray/tune/hist_stats/_robot_coordinates=[[array([2.22182286, 0.96829535]), array([2.22182286, 0.96829535]), array([2.22182286, 0.96829535]), array([2.22243168, 0.97018138]), array([2.22403719, 0.97485714]), array([2.22631157, 0.98168092]), array([2.22878956, 0.99026864]), array([2.23132091, 1.00169847]), array([2.23335957, 1.01504831]), array([2.23439554, 1.03005538]), array([2.23435593, 1.04599622]), array([2.23363013, 1.06344929]), array([2.2322367 , 1.08161665]), array([2.22986788, 1.10022734]), array([2.22665446, 1.12003667]), array([2.22300112, 1.14064891]), array([2.21929459, 1.16074016]), array([2.2150253 , 1.18057723]), array([2.20996839, 1.19983985]), array([2.20416782, 1.21784156]), array([2.1979178 , 1.23575665]), array([2.19129402, 1.254735  ]), array([2.18501007, 1.27370891]), array([2.17987784, 1.29186731]), array([2.17579357, 1.31074971]), array([2.17232378, 1.33154986]), array([2.169

Trial name,status,loc,reward,total time (s),ts,iter
PPO,RUNNING,152.66.246.136:3403,9.88353,410.664,12720,3


Trial name,status,loc,reward,total time (s),ts,iter
PPO,TERMINATED,,9.88353,410.664,12720,3


2021-07-13 08:19:10,935	INFO tune.py:352 -- Returning an analysis object by default. You can call `analysis.trials` to retrieve a list of trials. This message will be removed in future versions of Tune.


# 3. Testing


In [ ]:
checkpoint_path = "artifacts/ppo_DA/PPO_0_2021-07-12_11-08-383cftwzpe/checkpoint_24/checkpoint-24"

ppo_config["env_config"]["mode"] = 'inference'
ppo_config["env_config"]["domain_rand"] = False



In [ ]:
ppo_config

In [ ]:
# Restore agent
trainer = PPOTrainer(config=ppo_config)
trainer.restore(checkpoint_path)